### Streaming 응답 예제

In [ ]:
from openai import OpenAI
from src.constants import OPENAI_APIKEY
client = OpenAI(api_key=OPENAI_APIKEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Hello, who are you and what do you do?"
        }
    ]
)

print(completion.choices[0].message.content)

Hello! I’m an AI language model created by OpenAI. My main purpose is to assist users by providing information, answering questions, generating text, and engaging in conversation on a wide range of topics. Whether you need help with learning something new, solving a problem, or just want to chat, I'm here to help! What can I assist you with today?


In [2]:
print(completion.choices[0].message.content)

Function calls itself,  
Layers of depth intertwine,  
Infinity's loop.


In [ ]:
completion = client.chat.completions.create(
      model="Mistral-Nemo-12b",
      messages=[
        {"role": "system", "content": "You are a helpful assistant. Always answer in Korean."},
        {"role": "user", "content": "OpenAI에 대해 알려줄래?"}
        ],
      stream=False
)

print(completion.choices[0].message.content)

In [ ]:
completion.choices[0].message.content

In [ ]:
import requests

url = "http://localhost:8080/v1/chat/completions"
payload = {
    "messages": [{"role": "user", "content": "Hello, how are you?"}],
    "stream": True
}

response = requests.post(url, json=payload, stream=True)

for line in response.iter_lines():
    if line.detail:
        print(line.decode('utf-8'))


In [ ]:
import uvicorn
import argparse
import torch
from threading import Thread
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [ ]:
MODEL_PATH="/Users/ckchoi/Desktop/chatbot/Mistral-Nemo-12b"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )

In [ ]:
messages=[
        {"role": "user", "content": "안녕?"}
    ]

In [ ]:
def generate_response(messages):
    inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    response = ''
    print(f"streamer: {streamer}")
    for text in streamer:
        response += text + ' '
        print(f"text: {text}")
        yield response

In [ ]:
generate_response(messages)

In [ ]:
for chunk in generate_response(messages):
    print(f"chunk: {chunk}")

In [ ]:
inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
for text in streamer:
    print(type(text))